In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load
NYC1_data_to_load = "resources/NYC_Listings1.csv"
NYC2_data_to_load = "resources/NYC_Listings2.csv"
NYC3_data_to_load = "resources/NYC_Listings3.csv"

# Read School and Student Data File and store into Pandas Data Frames
NYC1_data = pd.read_csv(NYC1_data_to_load)
NYC2_data = pd.read_csv(NYC2_data_to_load)
NYC3_data = pd.read_csv(NYC3_data_to_load)

/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# Create a full NYC DF
NYC_data = pd.concat([NYC1_data, NYC2_data, NYC3_data], ignore_index=True)
NYC_data

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,2.019080e+13,8/7/19,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2.0,1.0,0.0,1.0,0.39
1,3647,https://www.airbnb.com/rooms/3647,2.019080e+13,8/6/19,THE VILLAGE OF HARLEM....NEW YORK !,NaN,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,0.0,1.0,0.0,NaN
2,3831,https://www.airbnb.com/rooms/3831,2.019080e+13,8/6/19,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1.0,1.0,0.0,0.0,4.64
3,5022,https://www.airbnb.com/rooms/5022,2.019080e+13,8/6/19,Entire Apt: Spacious Studio/Loft by central park,NaN,Loft apartment with high ceiling and wood floo...,Loft apartment with high ceiling and wood floo...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.10
4,5099,https://www.airbnb.com/rooms/5099,2.019080e+13,8/6/19,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment is true New York ...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment is true New York ...,none,My neighborhood in Midtown East is called Murr...,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.60
5,5121,https://www.airbnb.com/rooms/5121,2.019080e+13,8/7/19,BlissArtsSpace!,NaN,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,1.0,0.0,1.0,0.0,0.39
6,5178,https://www.airbnb.com/rooms/5178,2.019080e+13,8/6/19,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"You will use one large, furnished, private roo...",Please don’t expect the luxury here just a bas...,none,"Theater district, many restaurants around here.",...,f,f,strict_14_with_grace_period,f,f,1.0,0.0,1.0,0.0,3.48
7,5203,https://www.airbnb.com/rooms/5203,2.019080e+13,8/6/19,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",Stay in my family's little guest room and enj...,"Our best guests are seeking a safe, clean, spa...",none,Our neighborhood is full of restaurants and ca...,...,f,f,flexible,t,t,1.0,0.0,1.0,0.0,0.98
8,5222,https://www.airbnb.com/rooms/5222,2.019080e+13,8/6/19,Best Hideaway,NaN,"Sunny, quiet 1-bedroom in the heart of the Eas...","Sunny, quiet 1-bedroom in the heart of the Eas...",none,"The East Village is a fun, romantic, artists' ...",...,f,f,strict_14_with_grace_period,f,f,1.0,1.0,0.0,0.0,0.74
9,5238,https://www.airbnb.com/rooms/5238,2.019080e+13,8/6/19,Cute & Cozy Lower East Side 1 bdrm,"Live like a New Yorker instead of a stuffy, ti...",PLEASE NOTE: do not automatically book with ou...,"Live like a New Yorker instead of a stuffy, ti...",none,"Historically, the Lower East Side was a stoppi...",...,f,f,strict_14_with_grace_period,f,f,4.0,2.0,0.0,2.0,1.33


In [18]:
# Review columns
column_names = list(NYC_data.columns)
column_names

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [19]:
# Select Columns for clean DataFrame
NYC_data['id']
NYC_data['name']
NYC_data['host_location']
NYC_data['latitude']
NYC_data['longitude']
NYC_data['property_type']
NYC_data['room_type']
NYC_data['accommodates']
NYC_data['amenities']
NYC_data['price']
NYC_data['availability_365']
NYC_data['number_of_reviews']
NYC_data['number_of_reviews_ltm']
NYC_data['review_scores_rating']
NYC_data['review_scores_location']
NYC_data['reviews_per_month']

Clean_NYC = NYC_data[['id','name','neighbourhood',
                      'latitude','longitude','property_type',
                      'room_type','accommodates','amenities',
                      'price','availability_365','number_of_reviews',
                     'number_of_reviews_ltm','review_scores_rating',
                      'review_scores_location','reviews_per_month']]
# Drop null values
Clean_NYC = Clean_NYC.dropna()
# Preview DataFrame
Clean_NYC

,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month
0,2595,Skylit Midtown Castle,Midtown,40.75362,-73.98377,Apartment,Entire home/apt,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",$225.00,288.0,46.0,12.0,95.0,10.0,0.39
2,3831,Cozy Entire Floor of Brownstone,Brooklyn,40.68514,-73.95976,Guest suite,Entire home/apt,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$89.00,212.0,274.0,70.0,90.0,10.0,4.64
3,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,40.79851,-73.94399,Apartment,Entire home/apt,1.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Elev...",$80.00,0.0,9.0,2.0,93.0,9.0,0.10
4,5099,Large Cozy 1 BR Apartment In Midtown East,Midtown East,40.74767,-73.97500,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Buzzer/w...",$200.00,127.0,75.0,10.0,89.0,9.0,0.60
5,5121,BlissArtsSpace!,Brooklyn,40.68688,-73.95596,Apartment,Private room,2.0,"{Wifi,""Air conditioning"",Kitchen,""Pets live on...",$60.00,0.0,49.0,0.0,90.0,9.0,0.39
6,5178,Large Furnished Room Near B'way,Manhattan,40.76489,-73.98493,Apartment,Private room,2.0,"{TV,Wifi,""Air conditioning"",""Paid parking off ...",$79.00,239.0,434.0,44.0,84.0,10.0,3.48
7,5203,Cozy Clean Guest Room - Family Apt,Upper West Side,40.80178,-73.96723,Apartment,Private room,1.0,"{Internet,Wifi,""Air conditioning"",""Paid parkin...",$79.00,0.0,118.0,0.0,98.0,10.0,0.98
8,5222,Best Hideaway,Manhattan,40.72764,-73.97949,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$116.00,322.0,94.0,0.0,95.0,10.0,0.74
9,5238,Cute & Cozy Lower East Side 1 bdrm,Lower East Side,40.71344,-73.99037,Apartment,Entire home/apt,3.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",$150.00,0.0,161.0,13.0,94.0,9.0,1.33
10,5295,Beautiful 1br on Upper West Side,Manhattan,40.80316,-73.96545,Apartment,Entire home/apt,2.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Door...",$135.00,1.0,53.0,14.0,93.0,9.0,0.42


In [23]:
# GroupBY Neighborhood
Clean_NYC.groupby('neighbourhood').describe()

accommodates                                    \
                                     count       mean       std   min    25%   
neighbourhood                                                                  
Allerton                              10.0   2.900000  0.994429   2.0   2.00   
Alphabet City                        193.0   2.616580  1.495874   1.0   2.00   
Annadale                               2.0   3.500000  0.707107   3.0   3.25   
Arrochar                               1.0   4.000000       NaN   4.0   4.00   
Astoria                              470.0   2.508511  1.608523   1.0   2.00   
Bath Beach                             3.0   3.333333  2.309401   2.0   2.00   
Battery Park City                      8.0   1.875000  0.991031   1.0   1.00   
Bay Ridge                             77.0   3.233766  2.258878   1.0   2.00   
Baychester                            21.0   3.142857  2.220039   1.0   2.00   
Bayside                               18.0   2.388889  1.334558   1.0   1.25   
Bedford Park                          10.0   2.800000  1.813529   1.0   2.00   
Bedford-Stuyvesant                  1393.0   3.010768  2.050518   1.0   2.00   
Belmont                                4.0   1.000000  0.000000   1.0   1.00   
Bensonhurst                           27.0   2.925926  1.998575   1.0   2.00   
Bergen Beach                           2.0   4.000000  0.000000   4.0   4.00   
Boerum Hill                           76.0   3.144737  2.108421   1.0   2.00   
Borough Park                           8.0   2.125000  0.834523   1.0   2.00   
Brighton Beach                        18.0   3.166667  1.504894   1.0   2.00   
Bronxdale                              4.0   2.250000  1.258306   1.0   1.75   
Brooklyn                            7482.0   2.929431  1.907945   1.0   2.00   
Brooklyn Heights                      29.0   2.862069  1.552053   1.0   2.00   
Brooklyn Navy Yard                    25.0   2.920000  1.705872   1.0   2.00   
Brownsville                           14.0   2.785714  1.251373   1.0   2.00   
Bushwick                             693.0   2.597403  1.653808   1.0   2.00   
Canarsie                              63.0   3.523810  2.481266   1.0   2.00   
Carroll Gardens                       64.0   3.328125  1.894476   1.0   2.00   
Castle Hill                            2.0   2.500000  0.707107   2.0   2.25   
Castleton Corners                      1.0   8.000000       NaN   8.0   8.00   
Chelsea                              336.0   2.669643  1.380385   1.0   2.00   
Chinatown                            100.0   2.850000  1.737146   1.0   2.00   
...                                    ...        ...       ...   ...    ...   
Sunset Park                           92.0   2.489130  1.402467   1.0   2.00   
The Bronx                            382.0   3.107330  2.591425   1.0   2.00   
The Rockaways                        143.0   3.867133  2.551547   1.0   2.00   
Throgs Neck                            7.0   3.142857  1.676163   1.0   2.00   
Times Square/Theatre District         19.0   2.473684  1.263523   1.0   2.00   
Todt Hill                              1.0   3.000000       NaN   3.0   3.00   
Tompkinsville                          7.0   4.428571  3.359422   1.0   2.50   
Tottenville                            1.0  11.000000       NaN  11.0  11.00   
Tremont                               13.0   3.461538  3.596651   1.0   1.00   
Tribeca                               34.0   3.147059  2.076307   1.0   2.00   
Union Square                           2.0   3.000000  1.414214   2.0   2.50   
University Heights                     6.0   2.166667  0.983192   1.0   1.25   
Upper East Side                      528.0   2.672348  1.390068   1.0   2.00   
Upper West Side                      564.0   2.835106  1.574477   1.0   2.00   
Utopia                                 9.0   2.000000  0.000000   2.0   2.00   
Van Nest                               2.0   1.000000  0.000000   1.0   1.00   
Wakefield                     